In [4]:
import pandas as pd
import numpy as np
import transformers

In [5]:
data = pd.read_csv('EarningCallData/output.csv')

In [7]:
model_name = 'ProsusAI/finbert'
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

test = " I am so happy 🙂"
print(model(tokenizer(test, return_tensors='pt'))[0])

NameError: name 'transformers' is not defined

In [ ]:
def get_sentiment_bert(data):
    pass

In [ ]:
words = ['margin', 'cost', 'revenue', 'earnings', 'growth', 'debt', 'dividend', 'cashflow']